## This code loads a test initial condition and predicts the system state from that initial condition for a specified number of time-points into the future using the deepDMD model.

In [ ]:
import scipy.io as sio
import tensorflow as tf
import numpy as np
import scipy
import os 
import matplotlib.pyplot as plt
from time import process_time
from SM_wo_control import SurrogateModel

# Save Dir name
dir_name = 'Models/'

# Call the Surrogate Model. 
SM = SurrogateModel()

In [ ]:
# Mention the scenario.
test_scen = 410
filepath='../../Data/Test/scenario1/PMUData.mat'
mat_file_data = sio.loadmat(filepath)
o_f_data  = mat_file_data['PMU']['f'][0][0]
o_Vm_data = mat_file_data['PMU']['Vm'][0][0]
f_data  = 2*np.pi*(o_f_data-60)
Vm_data = 10*(o_Vm_data - 1)

start_point = 101
end_point   = start_point+1
# Collect the initial condition. 
X0 = np.concatenate((f_data[range(start_point,end_point)], Vm_data[range(start_point,end_point)]), axis = 1)
# Number of time steps prediction is needed. 
num_time_steps = 500
# Predict the initial condition. 
X_pred = SM.predict(X0, num_time_steps)
# Plot the predictions. 
data_window = np.multiply(range(end_point,end_point+num_time_steps),0.02)
# plt.plot(data_window,o_f_data[range(end_point,end_point+num_time_steps)],'b')
# plt.plot(data_window,60+(1/(2*np.pi))*X_pred[:,:68],'r--',linewidth = 2)
chose_buses = np.arange(0,67) #[1, 28, 35, 63, 67]

fig, (ax1, ax2) = plt.subplots(2,1)

ax1.plot(data_window,o_f_data[end_point:end_point+num_time_steps,chose_buses],'b')
ax1.plot(data_window,60+(1/(2*np.pi))*X_pred[:,chose_buses],'r--',linewidth = 2)
ax1.set_title('On the State space - Scenario ' + str(test_scen))
ax1.set_ylabel('Frequency [Hz]')

# plt.plot(data_window,o_Vm_data[range(end_point,end_point+num_time_steps)],'b')
# plt.plot(data_window,1+0.1*X_pred[:,68:136],'r--',linewidth = 2)
ax2.plot(data_window,o_Vm_data[end_point:end_point+num_time_steps,chose_buses],'b')
ax2.plot(data_window,1+0.1*X_pred[:,[x1+68 for x1 in chose_buses]],'r--',linewidth = 2)
# ax2.set_title('On the State space - Scenario ' + str(test_scen))
ax2.set_xlabel('Time')
ax2.set_ylabel('Voltage Magnitude [pu]')
plt.savefig(dir_name+"Test_prediction_deepDMD_Scenario_" + str(test_scen)".png", dpi=300)
plt.show()